In [4]:
import pandas as pd 
import numpy as np
import requests 
from bs4 import BeautifulSoup
import pickle
import gc
import multiprocessing
from multiprocessing import Pool
from time import sleep
import requests
import re
from function import get_company_info, get_list, unique_list # Save a function to 
import itertools

In [5]:
%%time
if __name__ ==  '__main__':
    
    # Get a list of Websites
    # First page
    websites = ['https://mrolinks.mro-network.com/exhibitordirectory?product_categories=20821&countryfield=US'] # First Page
    # Get the rest of the web pages 
    for i in range(1,87):
        web = 'https://mrolinks.mro-network.com/exhibitordirectory?page=' +str(i) +'&product_categories=20821&countryfield=US'
        websites.append(web)
        
    # Define a list of zipcode 
    miles40 = [98134,98106,98126,98161,98104,98174,98144,98108,98164,98154,98111,98114,98124,98129,98138,98145,
           98170,98181,98185,98190,98191,98113,98127,98139,98141,98165,98175,98194,98101,98116,98121,98122,
           98136,98118,98109,98102,98112,98040,98119,98146,98039,98195,98168,98199,98178,98105,98004,98107,
           98103,98009,98015,98062,98131,98056,98115,98005,98117,98158,98166,98386,98148,98057,98006,98188,
           98033,98007,98083,98353,98384,98061,98125,98055,98110,98008,98366,98034,98059,98133,98070,98177,
           98378,98073,98052,98031,98032,98155,98198,98028,98310,98082,98013,98011,98160,98337,98058,98064,
           98035,98089,98075,98314,98074,98041,98311,98029,98359,98393,98392,98043,98342,98030,98345,98053,
           98367,98036,98020,98021,98072,98027,98322,98046,98042,98077,98370,98003,98026,98001,98023,98037,
           98063,98071,98093,98332,98002,98383,98346,98038,98395,98050,98312,98422,98012,98315,98407,98087,
           98014,98421,98417,98403,98047,98402,98092,98354,98329,98296,98416,98413,98401,98411,98412,98415,
           98431,98464,98471,98481,98493,98497,98448,98025,98204,98406,98364,98424,98405,98335,98275,98208,
           98010,98465,98419,98418,98051,98372,98404,98390,98371,98443,98466,98409,98352,98340,98024,98490,
           98065,98291,98333,98203,98408,98380,98394,98236,98467,98019,98349,98365,98391,98496,98524,98207,
           98528,98213,98499,98376,98373,98272,98201,98290,98447,98444,98498,98206,98398,98445,98388,98446,
           98374,98438,98439,98385,98351,98375,98260,98588,98546,98249,98430,98303,98270,98293,98325,98396,
           98258,98387,98339,98271,98327,98433,98516,98358,98045,98253,98592,98338,98251,98555,98368]
    
    # Define argument for multiprocessing 
    num_processors = multiprocessing.cpu_count()
    p=Pool(processes = num_processors)
    
    # Multiprocessing: Get all individual company's website
    MRO = p.map(get_list,[web for web in websites])
    MRO = list(itertools.chain.from_iterable(MRO))  # Flatten the MRO list 
    print('Number of MRO: ', len(MRO))
    
    # Get Company info
    results = p.map(get_company_info, [web for web in MRO])
    company_names = [result[0] for result in results]
    zipcodes = [result[1] for result in results]
    
    # Return a Data Frame 
    df = pd.DataFrame({'MRO': company_names,
                  'zipcode': zipcodes})
    
    # Check if the MRO company are within range 
    df['Within_Range'] = df['zipcode'].isin(miles40).astype(int)
    
    # Save an CSV file 
    df.to_csv('MRO_List.csv', index = False)

Number of MRO:  1706


IndexError: list index out of range

In [6]:
df = pd.read_csv('MRO_List.csv', header = 0)
df.head()

,Unnamed: 0,MRO,zipcode,30miles?
0,1489,SpanaFlight,98374,1.0
1,540,"Avian Aeronautics, Inc.",98367,1.0
2,1474,"Soundair Repair Group, LLC",98290,1.0
3,671,Cannon Aircraft Interiors-Cai Avionics,98204,1.0
4,866,Esterline,98204,1.0


In [7]:
df.shape

(1727, 4)